In [155]:
import pandas as pd
import os
import plotly
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import datetime

nhs_colours = ['#00A499', '#FFB81C', '#7C2855']

# Appointments Analysis

### What type of appointment did you get?

In [156]:
# import csv file
appointment_types = pd.read_csv('data/Appoinment_types.csv', sep=',', header=0)
# rename blank first col
appointment_types.rename({' ':'Appointemnt Types'}, axis=1, inplace=True)
appointment_types

,Appointemnt Types,Q1 2018,Q1 2019,Q1 2020,Q1 2021
0,To speak to someone on the phone,58654,60756,59375,309311
1,To speak to someone online (for example on a v...,393,596,970,17047
2,To see someone at another general practice loc...,20082,24930,26545,16622
3,To see someone at my GP practice,548877,549943,514151,313459
4,For a home visit,5351,4944,4451,3228


In [157]:
# create a dict of appointments and there coresponding type
grouptype = {
    'To speak to someone on the phone': 'Remote',
    'To speak to someone online (for example on a video call)': 'Remote',
    'To see someone at another general practice location': 'Face-to-face',
    'To see someone at my GP practice': 'Face-to-face',
    'For a home visit': 'Face-to-face'
    }
# make a new col that maps the groups to the appointments
appointment_types['Group Type'] = appointment_types['Appointemnt Types'].map(grouptype)
appointment_types

,Appointemnt Types,Q1 2018,Q1 2019,Q1 2020,Q1 2021,Group Type
0,To speak to someone on the phone,58654,60756,59375,309311,Remote
1,To speak to someone online (for example on a v...,393,596,970,17047,Remote
2,To see someone at another general practice loc...,20082,24930,26545,16622,Face-to-face
3,To see someone at my GP practice,548877,549943,514151,313459,Face-to-face
4,For a home visit,5351,4944,4451,3228,Face-to-face


In [158]:
# now we can groupby type and sum the yearly counts
appointment_types_group = appointment_types.groupby('Group Type').sum().reset_index()
# melt is like an inverse pivot, changing the data to 'long' format (one observeation per row)
appointment_types_group = pd.melt(
    appointment_types_group,
    id_vars='Group Type',
    value_vars=[
        'Q1 2018',
        'Q1 2019',
        'Q1 2020',
        'Q1 2021'
        ]
    )
appointment_types_group.columns = ["Appointment Type", "Year", "Count"]
appointment_types_group

,Appointment Type,Year,Count
0,Face-to-face,Q1 2018,574310
1,Remote,Q1 2018,59047
2,Face-to-face,Q1 2019,579817
3,Remote,Q1 2019,61352
4,Face-to-face,Q1 2020,545147
5,Remote,Q1 2020,60345
6,Face-to-face,Q1 2021,333309
7,Remote,Q1 2021,326358


In [159]:
# use groupby on years to get the total counts
appointment_year_total = appointment_types_group.groupby('Year').sum().reset_index()
appointment_year_total.rename(columns={'Count': 'Total'}, inplace=True)
appointment_year_total 

,Year,Total
0,Q1 2018,633357
1,Q1 2019,641169
2,Q1 2020,605492
3,Q1 2021,659667


In [160]:
# merge the totals back to the df
appointment_types_group = pd.merge(appointment_types_group, appointment_year_total, on='Year', how='left')
# calculate percentages
appointment_types_group['Percent'] = appointment_types_group['Count']/appointment_types_group['Total']
appointment_types_group

,Appointment Type,Year,Count,Total,Percent
0,Face-to-face,Q1 2018,574310,633357,0.906771
1,Remote,Q1 2018,59047,633357,0.093229
2,Face-to-face,Q1 2019,579817,641169,0.904312
3,Remote,Q1 2019,61352,641169,0.095688
4,Face-to-face,Q1 2020,545147,605492,0.900337
5,Remote,Q1 2020,60345,605492,0.099663
6,Face-to-face,Q1 2021,333309,659667,0.505269
7,Remote,Q1 2021,326358,659667,0.494731


### Appointments: data visualisation

In [161]:
# plot the long (tidy) dataframe
fig = px.bar(
    appointment_types_group,
    x="Year",
    y="Percent",
    color="Appointment Type",
    color_discrete_sequence=nhs_colours,
    # I would use the website to write chart titles in the correct font/foramt/spacing
    #title="Remote appointments are now as common as in person appointments",
    barmode="stack",
    text="Percent",
    height=350,
)

# Asthetics of the plot
fig.update_layout(
    # change background colour 
    {"plot_bgcolor": "rgba(0, 0, 0, 0)", "paper_bgcolor": "rgba(0, 0, 0, 0)"},
    yaxis_title='Of survey respondents (%)',
    yaxis=dict(tickformat = "%"),
    xaxis_title=None,
    # change fonts
    font=dict(family="Arial", size=16),
    # autosize makes the chart 'responsive' (https://en.wikipedia.org/wiki/Responsive_web_design)
    autosize=True,
    margin=dict(
        l=50,
        r=50,
        b=50,
        t=50,
        pad=4,
        autoexpand=True
        ),
    ## options for the hover-over text
    # changes how the hover-over box is displayed
    hovermode="x unified",
    # hoverlabel_align = 'right',
    # changes the hover-over box formatting
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="Arial"
    ),
    legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
    )
)

# options to change how the bars look
fig.update_traces(
    # this also changes the hover-over box formatting
    hovertemplate='<b>%{fullData.name}</b>: %{text:.1%}<extra></extra>',
    # outline of bars
    marker_line_color = 'rgb(8,48,107)',
    marker_line_width=1.5,
    opacity=1,
    # options for text inside bars
    texttemplate='%{text:.1%}',
    textposition='inside'
)
fig.update_xaxes(type='category')
fig.show()

In [162]:
# Write chart to file (.html)
config = {"displayModeBar": False, "displaylogo": False}
plotly_obj = plotly.offline.plot(
    fig, include_plotlyjs=False, output_type="div", config=config
)
with open("_includes/remote_appts.html", "w") as file:
    file.write(plotly_obj)

html_str = '<b>Figure 1.</b> Proportion of appointment types betweeen survey respondents, where Remote includes both Telephone and Online appointments. Source: GP Patient Survey 2021.'

In [163]:
## how did you try to book an appointment?
how_try_book_appointment = pd.read_csv(r'data/how_book_appointment.csv', sep=',', header=0)
how_try_book_appointment = how_try_book_appointment.rename(columns={
    "type": "Booking Type",
    'National_N': "Count"})

# create a dict of appointments and there coresponding type
grouptype = {
    'In person': 'In person',
    'By phone, through my practice': 'Telephone',
    'By automated telephone booking': 'Telephone',
    'Online, including on a website or through an app': 'Online'
    }
# make a new col that maps the groups to the appointments
how_try_book_appointment['Group Type'] = how_try_book_appointment['Booking Type'].map(grouptype)

# now we can groupby type and sum the yearly counts
how_try_book_appointment_group = how_try_book_appointment.groupby('Group Type').sum().reset_index()

# sum series to get total count
how_try_book_appointmen_total = how_try_book_appointment_group.sum(axis=0).values[1]

# calculate percentages
how_try_book_appointment_group['Percent'] = how_try_book_appointment_group['Count']/how_try_book_appointmen_total
how_try_book_appointment_group


,Group Type,Count,Percent
0,In person,65700,0.076663
1,Online,105510,0.123116
2,Telephone,685789,0.800221


In [164]:
fig = px.pie(how_try_book_appointment_group, color_discrete_sequence=nhs_colours, values='Percent', names='Group Type')

# Asthetics of the plot
fig.update_layout(
    # change background colour 
    {"plot_bgcolor": "rgba(0, 0, 0, 0)", "paper_bgcolor": "rgba(0, 0, 0, 0)"},
    # change fonts
    font=dict(family="Arial", size=16),
    # autosize makes the chart 'responsive' (https://en.wikipedia.org/wiki/Responsive_web_design)
    autosize=True,
    margin=dict(
        l=50,
        r=50,
        b=50,
        t=50,
        pad=4,
        autoexpand=True
        ),
    ## options for the hover-over text
    # changes how the hover-over box is displayed
    # hoverlabel_align = 'right',
    # changes the hover-over box formatting
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="Arial"
    ),
    legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
    )
)
# options to change how the bars look
fig.update_traces(
    # this also changes the hover-over box formatting
    hovertemplate='<b>%{label}</b>: %{value:.1%}',
    # outline of bars
    marker_line_color = 'rgb(8,48,107)',
    marker_line_width=1.5,
    opacity=1,
    # options for text inside bars
    textposition='inside'
)


fig.show()

In [165]:
# Write chart to file (.html)
config = {"displayModeBar": False, "displaylogo": False}
plotly_obj = plotly.offline.plot(
    fig, include_plotlyjs=False, output_type="div", config=config
)
with open("_includes/booking_type.html", "w") as file:
    file.write(plotly_obj)

# GP online services

### Which of the following general practice online services have you used in the past 12 months?

In [166]:
# Could you change the file to data/Online_gp_services.csv
online_gp_services = pd.read_csv (r'data/Online_gp_services.csv', sep=',', header=0).dropna(axis='columns')
# remove those respondents that did not use any online service
online_gp_services = online_gp_services.rename(columns={
    "75_Over_N": "Aged 75+",
    '16_24_N': "Aged 16-24",
    'National_N': "All Patients"})
online_gp_services = pd.melt(
    online_gp_services,
    id_vars=['Activity', 'Lable'],
    value_vars=[
        "Aged 75+",
        "Aged 16-24",
        "All Patients"])
online_gp_services.columns = ["Online Activity", "Lable", "Age Group", "Count of Respondents"]

# use groupby on age to get the total counts
online_gp_services_total = online_gp_services.groupby('Age Group').sum().reset_index()
online_gp_services_total.rename(columns={'Count of Respondents': 'Total Respondents'}, inplace=True)

# merge the totals back to the df
online_gp_services = pd.merge(online_gp_services, online_gp_services_total, on='Age Group', how='left')
# calculate percentages
online_gp_services['Percent'] = online_gp_services['Count of Respondents']/online_gp_services['Total Respondents']
online_gp_services = online_gp_services.sort_values(['Percent'], ascending=True)
online_gp_services = online_gp_services[~online_gp_services['Online Activity'].str.contains("None of these")]
online_gp_services

,Online Activity,Lable,Age Group,Count of Respondents,Total Respondents,Percent
2,Accessing my medical records online,Access medical<br>record,Aged 75+,2653,97009,0.027348
12,Accessing my medical records online,Access medical<br>record,All Patients,58910,1046278,0.056304
7,Accessing my medical records online,Access medical<br>record,Aged 16-24,6166,104776,0.058849
0,Booking appointments online,Book an<br>appointment,Aged 75+,8710,97009,0.089785
3,Had an online consultation or appointment,Had a Consultation<br>or appointment,Aged 75+,9408,97009,0.096981
6,Ordering repeat prescriptions online,Order repeat<br>prescription,Aged 16-24,14599,104776,0.139335
13,Had an online consultation or appointment,Had a Consultation<br>or appointment,All Patients,148409,1046278,0.141845
10,Booking appointments online,Book an<br>appointment,All Patients,155845,1046278,0.148952
5,Booking appointments online,Book an<br>appointment,Aged 16-24,16363,104776,0.156171
8,Had an online consultation or appointment,Had a Consultation<br>or appointment,Aged 16-24,16590,104776,0.158338


In [167]:

# plot the long (tidy) dataframe
fig = px.bar(
    online_gp_services,
    x="Online Activity",
    y="Percent",
    color="Age Group",
    color_discrete_sequence=nhs_colours,
    barmode="group",
    text="Percent",
    height=350,
)

# Asthetics of the plot
fig.update_layout(
    # change background colour 
    {"plot_bgcolor": "rgba(0,0,0, 0)", "paper_bgcolor": "rgba(0,0,0, 0)"},
    yaxis_title='Of survey respondents (%)',
    yaxis=dict(
        tickformat = "%",
        range=[0, 0.3]),
    xaxis=dict(
        tickmode = 'array',
        tickvals = online_gp_services['Online Activity'].tolist(),
        ticktext = online_gp_services['Lable'].tolist(),
        #tickangle = 0
    ),
    xaxis_title=None,
    # change fonts
    font=dict(family="Arial", size=16),
    # autosize makes the chart 'responsive' (https://en.wikipedia.org/wiki/Responsive_web_design)
    autosize=True,
    margin=dict(
        l=50,
        r=50,
        b=50,
        t=50,
        pad=4,
        autoexpand=True
        ),
    ## options for the hover-over text
    # changes how the hover-over box is displayed
    hovermode="x unified",
    # hoverlabel_align = 'right',
    # changes the hover-over box formatting
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="Arial"
    ),
    legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
    )
)
# options to change how the bars look
fig.update_traces(
    # this also changes the hover-over box formatting
    hovertemplate='<b>%{fullData.name}</b>: %{text:.1%}<extra></extra>',
    # outline of bars
    marker_line_color = 'rgb(8,48,107)',
    marker_line_width=1.5,
    opacity=1,
    # options for text inside bars
    texttemplate='%{text:.1%}',
    textposition='inside'
    )
fig.update_xaxes(type='category')
fig.show()

In [168]:
# Write chart to file (.html)
config = {"displayModeBar": False, "displaylogo": False}
plotly_obj = plotly.offline.plot(
    fig, include_plotlyjs=False, output_type="div", config=config
)
with open("_includes/online_activities_all.html", "w") as file:
    file.write(plotly_obj)

html_str = '<b>Figure 2.</b> The proportion of different online service use, between survey respondents that used an online services within the last 12 months (as of March 2021). Source: GP Patient Survey 2021.'

## Trends - Using GP Websites

## How easy is it to use your GP practice’s website to look for information or access services?

In [169]:
# Over 75s (2018 - 2021)
ease_of_use_75 = pd.read_csv (r'data/Ease_of_use_over_time_75+.csv', header = 0)
ease_of_use_75.columns = ["Year", "Percent"]

# 16-24s (2018 - 2021)
ease_16_24 = pd.read_csv (r'data/use_16_24.csv', header = 0)                                                                                                                                                 
ease_16_24.columns = ["Year", "Age 16-24"]

# Long term health conditions (2018 - 2021)
trend_ease_use_LTC = pd.read_csv (r'data/trend_ease_use_2021_time_long_term_condition.csv')
trend_ease_use_LTC = trend_ease_use_LTC.dropna(axis='rows')
trend_ease_use_LTC.columns = ["Year", "Long-term Cond."]

# merge
#ease_of_use = ease_16_24.merge(ease_use_75_and_over,on='Year').merge(trend_ease_use_LTC,on='Year')
#ease_of_use = pd.melt(
#    ease_of_use,
#    id_vars=['Year'],
#    value_vars=[
#        "Age 16-24",
#        "Age 75+",
#        "Long-term Cond."])
#ease_of_use.columns = ["Year", "Group", "Percent"]
#ease_of_use

#ease_of_use_75 = ease_of_use[ease_of_use['Group']=='Age 75+']
#ease_of_use_75 = ease_of_use_75.sort_values(['Year'], ascending=True)
ease_of_use_75['Age Group'] = 'Age 75+'
ease_of_use_75['Difficult'] = 1-ease_of_use_75['Percent']
ease_of_use_75

,Year,Percent,Age Group,Difficult
0,March 2018,0.84,Age 75+,0.16
1,March 2019,0.81,Age 75+,0.19
2,March 2020,0.80,Age 75+,0.20
3,March 2021,0.75,Age 75+,0.25


In [170]:
# plot the long (tidy) dataframe
fig = px.bar(
    ease_of_use_75,
    x="Year",
    y="Difficult",
    color="Age Group",
    color_discrete_sequence=nhs_colours,
    barmode="group",
    text="Difficult",
    height=350,
)

# Asthetics of the plot
fig.update_layout(
    # change background colour 
    {"plot_bgcolor": "rgba(0, 0, 0, 0)", "paper_bgcolor": "rgba(0, 0, 0, 0)"},
    yaxis_title='Of survey respondents (%)',
    xaxis_title=None,
    yaxis=dict(
        tickformat = "%",
        range=[0, 0.3]),
    # change fonts
    font=dict(family="Arial", size=16),
    # autosize makes the chart 'responsive' (https://en.wikipedia.org/wiki/Responsive_web_design)
    autosize=True,
    margin=dict(
        l=50,
        r=50,
        b=50,
        t=0,
        pad=4,
        autoexpand=True
        ),
    ## options for the hover-over text
    # changes how the hover-over box is displayed
    hovermode="x unified",
    # hoverlabel_align = 'right',
    # changes the hover-over box formatting
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="Arial"
    ),
    legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
    )
)
# options to change how the bars look
fig.update_traces(
    # this also changes the hover-over box formatting
    hovertemplate='<b>%{fullData.name}</b>: %{text:.0%}<extra></extra>',
    # outline of bars
    marker_line_color = 'rgb(8,48,107)',
    marker_line_width=1.5,
    opacity=1,
    # options for text inside bars
    texttemplate='%{text:.0%}',
    textposition='inside',
    textangle = 0
    )
fig.update_xaxes(type='category')
fig.show()

In [171]:
# Write chart to file (.html)
config = {"displayModeBar": False, "displaylogo": False}
plotly_obj = plotly.offline.plot(
    fig, include_plotlyjs=False, output_type="div", config=config
)
with open("_includes/ease_of_use.html", "w") as file:
    file.write(plotly_obj)

html_str = '<b>Figure 1.</b> The proportion of survey respondents aged 75 and over that found it easy use their GP practice’s website to look for information or access services. Source: GP Patient Survey 2021.'

##  Which of the following general practice online services have you used in the past 12 months

In [172]:
# Q3. Which of the following general practice online services have you used in the past 12 months?
online_gp_services_LTC = pd.read_csv (r'data/GPPS_Long term health conditions 2021.csv', header = 0)
online_gp_services_LTC


,Activity,Lable,Condition,Count of Respondents
0,Booking appointments online,Book an<br>appointment,Long-term condition,78948
1,Ordering repeat prescriptions online,Order repeat<br>prescription,Long-term condition,148279
2,Accessing my medical records online,Access medical<br>record,Long-term condition,35153
3,Had an online consultation or appointment,Consultation or<br>appointment,Long-term condition,76365
4,None of these,None of these,Long-term condition,196402
5,Booking appointments online,Book an<br>appointment,No long-term condition,62795
6,Ordering repeat prescriptions online,Order repeat<br>prescription,No long-term condition,50856
7,Accessing my medical records online,Access medical<br>record,No long-term condition,19581
8,Had an online consultation or appointment,Consultation or<br>appointment,No long-term condition,59085
9,None of these,None of these,No long-term condition,213146


In [173]:
# remove those respondents that did not use any online service
online_gp_services_LTC = online_gp_services_LTC[~online_gp_services_LTC['Activity'].str.contains("None of these")]

# use groupby on age to get the total counts
online_gp_services_total = online_gp_services_LTC.groupby('Condition').sum().reset_index()
online_gp_services_total.rename(columns={'Count of Respondents': 'Total Respondents'}, inplace=True)
online_gp_services_total


,Condition,Total Respondents
0,Long-term condition,338745
1,No long-term condition,192317


In [174]:
# merge the totals back to the df
online_gp_services_LTC = pd.merge(online_gp_services_LTC, online_gp_services_total, on='Condition', how='left')
# calculate percentages
online_gp_services_LTC['Percent'] = online_gp_services_LTC['Count of Respondents']/online_gp_services_LTC['Total Respondents']
online_gp_services_LTC = online_gp_services_LTC.sort_values(['Percent', 'Condition'], ascending=True)
online_gp_services_LTC

,Activity,Lable,Condition,Count of Respondents,Total Respondents,Percent
6,Accessing my medical records online,Access medical<br>record,No long-term condition,19581,192317,0.101816
2,Accessing my medical records online,Access medical<br>record,Long-term condition,35153,338745,0.103774
3,Had an online consultation or appointment,Consultation or<br>appointment,Long-term condition,76365,338745,0.225435
0,Booking appointments online,Book an<br>appointment,Long-term condition,78948,338745,0.233060
5,Ordering repeat prescriptions online,Order repeat<br>prescription,No long-term condition,50856,192317,0.264438
7,Had an online consultation or appointment,Consultation or<br>appointment,No long-term condition,59085,192317,0.307227
4,Booking appointments online,Book an<br>appointment,No long-term condition,62795,192317,0.326518
1,Ordering repeat prescriptions online,Order repeat<br>prescription,Long-term condition,148279,338745,0.437730


In [175]:
# plot the long (tidy) dataframe
fig = px.bar(
    online_gp_services_LTC,
    x="Activity",
    y="Percent",
    color="Condition",
    color_discrete_sequence=nhs_colours,
    barmode="group",
    text="Percent",
    height=350,
)

# Asthetics of the plot
fig.update_layout(
    # change background colour 
    {"plot_bgcolor": "rgba(0,0,0, 0)", "paper_bgcolor": "rgba(0,0,0, 0)"},
    yaxis_title='Of respondents that use online services (%)',
    yaxis=dict(tickformat = "%"),
    xaxis=dict(
        tickmode = 'array',
        tickvals = online_gp_services_LTC['Activity'].tolist(),
        ticktext = online_gp_services_LTC['Lable'].tolist(),
        tickangle = 0
    ),
    xaxis_title=None,
    # change fonts
    font=dict(family="Arial", size=16),
    # autosize makes the chart 'responsive' (https://en.wikipedia.org/wiki/Responsive_web_design)
    autosize=True,
    margin=dict(
        l=50,
        r=50,
        b=50,
        t=50,
        pad=4,
        autoexpand=True
        ),
    ## options for the hover-over text
    # changes how the hover-over box is displayed
    hovermode="x unified",
    # hoverlabel_align = 'right',
    # changes the hover-over box formatting
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="Arial"
    ),
    legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
    )
)
# options to change how the bars look
fig.update_traces(
    # this also changes the hover-over box formatting
    hovertemplate='<b>%{fullData.name}</b>: %{text:.1%}<extra></extra>',
    # outline of bars
    marker_line_color = 'rgb(8,48,107)',
    marker_line_width=1.5,
    opacity=1,
    # options for text inside bars
    texttemplate='%{text:.1%}',
    textposition='inside',
    #textangle = 0
    )
fig.update_xaxes(type='category')
fig.show()

In [176]:
# Write chart to file (.html)
config = {"displayModeBar": False, "displaylogo": False}
plotly_obj = plotly.offline.plot(
    fig, include_plotlyjs=False, output_type="div", config=config
)
with open("_includes/online_activities_ltc.html", "w") as file:
    file.write(plotly_obj)

html_str = '<b>Figure 3.</b> The proportion of online service use between survey respondents (with and without long-term conditions) that used an online services within the last 12 months (as of March 2021). Source: GP Patient Survey 2021.'